In [ ]:
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Import des classes et fonctions depuis src
from src import (
    SERIES_DIR, TRAIN_CSV, TRAIN_LOCALIZERS_CSV, OUTPUT_DIR,
    print_config
)
from src.bricks import EDA, Preprocessor
from src import show_middle_slices

# Afficher la configuration
print("="*60)
print("Configuration de l'environnement")
print("="*60)
print_config()

# Exploration des Données DICOM

Ce notebook utilise la classe **EDA** pour analyser les données locales de manière systématique.

In [ ]:
# Charger les données
df_train = pd.read_csv(TRAIN_CSV)
df_localizers = pd.read_csv(TRAIN_LOCALIZERS_CSV)

print(f"📊 Données chargées:")
print(f"  - Séries totales: {len(df_train)}")
print(f"  - Localisateurs (avec anévrismes): {len(df_localizers)}")

In [ ]:
# Créer l'analyseur EDA
eda = EDA(df_train, df_localizers, SERIES_DIR)

print(f"✅ Analyseur EDA créé")
print(eda)

In [ ]:
# 1️⃣ Analyser les modalités
modality_counts = eda.analyze_modalities()

In [ ]:
# 2️⃣ Analyser la distribution des anévrismes
aneurysm_counts = eda.analyze_aneurysm_distribution()

In [ ]:
# 3️⃣ Analyser les positions anatomiques
position_counts = eda.analyze_positions()

In [ ]:
# 4️⃣ Visualiser les distributions
eda.plot_aneurysm_distribution()

In [ ]:
# 5️⃣ Détecter les séries défectueuses (optionnel - peut être long)
# Commentez cette cellule si vous avez beaucoup de séries

# defective_series = eda.detect_defective_series()
print("⚠️ Détection des séries défectueuses commentée (peut être long)")
print("Décommentez la cellule pour l'exécuter sur vos 20 séries locales")

In [ ]:
# 6️⃣ Analyser le nombre de slices
slice_stats = eda.analyze_slice_counts()

In [ ]:
# 7️⃣ Rapport complet
print("\n" + "="*60)
print("RAPPORT COMPLET")
print("="*60)
eda.generate_report()

In [ ]:
## Visualisation de Séries DICOM

Utilisons le **Preprocessor** pour charger et visualiser des volumes 3D.

In [ ]:
# Créer un preprocessor
preprocessor = Preprocessor(target_spacing=(0.4, 0.4, 0.4))

print("✅ Preprocessor créé")
print(preprocessor)

In [ ]:
# Sélectionner une série au hasard parmi les locales
import os
import glob

available_series = glob.glob(os.path.join(SERIES_DIR, '*'))

if available_series:
    # Prendre la première série disponible
    example_series = available_series[0]
    series_uid = os.path.basename(example_series)
    
    print(f"📁 Série sélectionnée: {series_uid}")
    print(f"📂 Chemin: {example_series}")
    
    # Compter les fichiers DICOM
    dicom_files = glob.glob(os.path.join(example_series, '*.dcm'))
    print(f"🔢 Nombre de slices: {len(dicom_files)}")
else:
    print("❌ Aucune série DICOM trouvée dans:", SERIES_DIR)

In [ ]:
# Charger et préprocesser le volume avec le Preprocessor
if available_series:
    print("🔄 Chargement et preprocessing du volume...")
    volume = preprocessor.process_volume(example_series)
    
    print(f"✅ Volume préprocessé:")
    print(f"  - Shape: {volume.shape}")
    print(f"  - Min: {volume.min():.4f}")
    print(f"  - Max: {volume.max():.4f}")
    print(f"  - Mean: {volume.mean():.4f}")
else:
    print("⚠️ Passez à la cellule suivante")

In [ ]:
# Visualiser les coupes du volume avec show_middle_slices
if available_series:
    print("📊 Visualisation des coupes centrales (axiale, coronale, sagittale):")
    show_middle_slices(volume)
else:
    print("⚠️ Aucun volume à visualiser")

In [ ]:
## Visualisation de Plusieurs Séries

Visualisons les 3 premières séries disponibles localement.

In [ ]:
# Visualiser les 3 premières séries
n_series_to_show = min(3, len(available_series))

for i in range(n_series_to_show):
    series_path = available_series[i]
    series_uid = os.path.basename(series_path)
    
    print(f"\n{'='*60}")
    print(f"Série {i+1}/{n_series_to_show}: {series_uid}")
    print(f"{'='*60}")
    
    try:
        # Preprocessing
        volume = preprocessor.process_volume(series_path)
        print(f"✅ Volume shape: {volume.shape}")
        
        # Visualisation
        show_middle_slices(volume)
        
    except Exception as e:
        print(f"❌ Erreur lors du traitement: {e}")

In [ ]:
## Conclusion

Ce notebook a utilisé les composants modulaires de `src/` :

### ✅ Classe EDA
- Analyse des modalités
- Distribution des anévrismes
- Positions anatomiques
- Détection de séries défectueuses
- Statistiques sur les slices

### ✅ Classe Preprocessor
- Chargement DICOM
- Resampling à espacement cible
- Cropping du fond
- Normalisation

### ✅ Fonctions de Visualisation
- `show_middle_slices()` pour visualiser les coupes 3D

### 🎯 Prochaines étapes

Consultez le notebook [02_dataset_creation.ipynb](02_dataset_creation.ipynb) pour créer un dataset d'entraînement à partir de ces données.

In [ ]:
df['Modality'].value_counts()

In [ ]:
df_MRA=df[df['Modality']=='MRA']
df_CTA=df[df['Modality']=='CTA']

In [ ]:
df_MRA=df_MRA[['SeriesInstanceUID', 'PatientPath', 'NumSlices', 'Aneurysm Present']]

In [ ]:
df_MRA['Aneurysm Present'].value_counts()

In [ ]:
df['Aneurysm Present'].value_counts()

In [ ]:
df_MRA.shape

In [ ]:
df_MRA['NumSlices'].hist(bins=100)
plt.xlabel("Nombre de slices")
plt.ylabel("Nombre de séries")
plt.show()

# Preprocessing des DICOM

In [ ]:
# Utiliser SERIES_DIR
series_path = SERIES_DIR
patient_list = glob.glob(series_path+'/*')

In [ ]:
# Transformer une série d'images DICOM en tableau 3D numpy
def dicom_to_numpy(patient_path):
    
    dicom_files = sorted(glob.glob(patient_path+'/*.dcm'))
    slices = [pydicom.dcmread(f) for f in dicom_files]
    #tri des slices par instance number
    slices.sort(key=lambda s: int(s.InstanceNumber))
    
    # On empile les pixel_array en un volume 3D NumPy (X,Y,Z)
    target_shape = slices[0].pixel_array.shape
    slices = [s for s in slices if s.pixel_array.shape == target_shape]
    volume = np.stack([s.pixel_array for s in slices], axis=-1)

    # Récupération du spacing réel
    pixel_spacing = slices[0].PixelSpacing
    dx, dy = pixel_spacing
    dz = getattr(slices[0], 'SliceThickness', 1.0)  # fallback si manquant
    
    return volume, (dx,dy,dz)

In [ ]:
def resample(volume, spacing):
    dx, dy, dz = spacing
    new_volume = zoom(volume, (dx/1, dy/1, dz/1), order=1)
    return new_volume

In [ ]:
def crop(volume):
    """
    Coupe le volume 3D pour ne garder que la région contenant du signal.
    threshold : valeurs < threshold sont considérées comme fond/noir.
    """
    # On crée un masque des voxels non nuls
    mask = volume > 10
    if not mask.any():
        return volume  # rien à couper
    
    # On récupère les indices min/max pour chaque dimension
    x_min, x_max = mask.any(axis=(1,2)).nonzero()[0][[0, -1]]
    y_min, y_max = mask.any(axis=(0,2)).nonzero()[0][[0, -1]]
    z_min, z_max = mask.any(axis=(0,1)).nonzero()[0][[0, -1]]
    
    # Crop
    cropped = volume[x_min:x_max+1, y_min:y_max+1, z_min:z_max+1]
    return cropped

In [ ]:
def resize(volume):
    """
    Redimensionne le volume 3D à la taille target_shape par interpolation linéaire.
    """
    target_shape=(128,128,64)
    factors = [t/s for t, s in zip(target_shape, volume.shape)]
    resized_volume = zoom(volume, factors, order=1)
    return resized_volume

In [ ]:
# Normaliser un tableau numpy 3D
def normalization(volume):
    """
    Normalise un volume entre 0 et 1 (par patient).
    """
    v_min, v_max = volume.min(), volume.max()
    if v_max > v_min:  # éviter la division par zéro
        volume = (volume - v_min) / (v_max - v_min)
    else:
        volume = np.zeros_like(volume)
    return volume

In [ ]:
def preprocessing(patient_path):
    
    volume,spacing = dicom_to_numpy(patient_path)
    volume = resample(volume, spacing)
    volume = crop(volume)
    volume = resize(volume)
    volume = normalization(volume)

    return volume

# Creer un Dataset PyTorch

In [ ]:
import torch
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

In [ ]:
class AneurysmDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)  # ton DataFrame avec PatientPath et label
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]                # récupère la ligne correspondante
        volume = preprocessing(row['PatientPath'])  # pipeline 3D complet
        label = row['Aneurysm Present']        # 0 ou 1

        # PyTorch attend (C, X, Y, Z) → ajouter une dimension channel
        volume = torch.tensor(volume, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor(label, dtype=torch.float32)
        return volume, label


In [ ]:
subset_train = df.iloc[:100]  # 50 patients pour train
subset_val = df.iloc[100:150] #10 patients pour validation

train_dataset=AneurysmDataset(subset_train)
val_dataset=AneurysmDataset(subset_val)

train_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=1,shuffle=False)

In [ ]:
df_train, df_val = train_test_split(df_MRA, test_size=0.2, 
                                                random_state=42, 
                                                stratify=df_MRA['Aneurysm Present'])

In [ ]:
train_dataset = AneurysmDataset(df_train)
val_dataset = AneurysmDataset(df_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ResNet 3D
On est pas encore sur un resnet en fait mais sur un CNN 3D

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score

from tqdm import trange
from tqdm import tqdm

In [ ]:
class SimpleResNet3D(nn.Module):
    def __init__(self):
        super(SimpleResNet3D, self).__init__()
        
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(16)
        
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, stride=2, padding=1)  # downsample
        self.bn2 = nn.BatchNorm3d(32)
        
        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm3d(64)
        
        self.conv4 = nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm3d(128)
        
        # Global average pooling → résumé du volume
        self.avgpool = nn.AdaptiveAvgPool3d(1)  
        
        # Classification binaire
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        
        x = self.avgpool(x)  # (B, 128, 1, 1, 1)
        x = torch.flatten(x, 1)  # (B, 128)
        x = self.fc(x)  # (B, 1)
        return torch.sigmoid(x)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SimpleResNet3D().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (volumes, labels) in enumerate(tqdm(train_loader)):
        volumes = volumes.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True).unsqueeze(1).float()  # (B,1)
        
        optimizer.zero_grad()
        outputs = model(volumes)  # déjà sigmoid
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * volumes.size(0)

        # Sauvegarde toutes les 10 itérations (par ex.)
        if batch_idx % 10 == 0:
            checkpoint_path = f"checkpoint_epoch{epoch}_batch{batch_idx}.pth"
            torch.save({
                'epoch': epoch,
                'batch_idx': batch_idx,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
            }, checkpoint_path)
        
    return running_loss / len(train_loader.dataset)

In [ ]:
def evaluate(model, val_loader, device):
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for volumes, labels in tqdm(val_loader):
            volumes = volumes.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True).unsqueeze(1).float()
            
            outputs = model(volumes)  # déjà sigmoïd
            preds = outputs  # proba
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    try:
        auc = roc_auc_score(all_labels, all_preds)
    except ValueError:
        auc = float('nan')  # si une seule classe dans val
    
    return auc

In [ ]:
# Charger un modèle sauvegardé localement
# Note: Adapter le chemin vers votre modèle local
# state_dict = torch.load(
#     "results/models/best_model_epoch3.pth",
#     map_location="cuda"
# )
# model.load_state_dict(state_dict)

print("⚠️ Cette cellule charge un modèle pré-entraîné depuis Kaggle.")
print("Commentez cette cellule si vous n'avez pas encore de modèle local.")

In [ ]:
# Tester avec un train et val loaders réduits
num_epochs = 4

best_auc = 0.5605  # pour sauvegarder le meilleur modèle

for epoch in range(3,num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_auc = evaluate(model, val_loader, device)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val AUC: {val_auc:.4f}")

    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), f"best_model_epoch{epoch+1}.pth")
        print(f"--> Modèle sauvegardé à l'epoch {epoch+1} avec AUC {val_auc:.4f}")


In [ ]:
checkpoint = torch.load("/default/1/checkpoint_epoch1_batch140.pth", map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

start_epoch = checkpoint['epoch']
start_batch = checkpoint['batch_idx'] + 1  # on repart juste après

In [ ]:
# Tester avec un train et val loaders réduits
num_epochs = 3

best_auc = 0.0  # pour sauvegarder le meilleur modèle

for epoch in range(2,num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_auc = evaluate(model, val_loader, device)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val AUC: {val_auc:.4f}")

    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), f"best_model_epoch{epoch+1}.pth")
        print(f"--> Modèle sauvegardé à l'epoch {epoch+1} avec AUC {val_auc:.4f}")